## Imports

In [1]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, DelimitedFiles, Combinatorics, NPZ

In [2]:
const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x0000000067d38ab0, false, 0)

In [3]:
TIME_LIMIT = 3600 * 24 * 10;

## Read in data, initialize parameters

In [4]:
NUM_CREWS = 10                
BREAK_LENGTH = 2       # how long at base to be considered "rested"

# tradeoffs
BETA = 100             # cost of one area unit burned / cost of mile traveled
ALPHA = 200            # cost of crew-day of suppression / cost of mile traveled
LINE_PER_CREW = 17     # how much perimeter prevented per crew per time period

# set path to all input data
in_path = "data/processed"

# get inital fire perimeters and no-suppression progression parameters
M = readdlm(in_path * "/sample_growth_patterns.csv", ',')
start_perims = M[:, 1]
progressions = M[:, 2:15]

NUM_TIME_PERIODS = size(M)[2] - 1 
NUM_FIRES = size(M)[1]       

# get distance from fire f to fire g 
fire_dists =  readdlm(in_path * "/fire_distances.csv", ',')

# get distance from base c to fire g (NUM_CREWS-by-NUM_FIRES)
base_fire_dists =  readdlm(in_path * "/base_fire_distances.csv", ',')

# initialize travel times (number of periods) from fire f to fire g
tau = convert(Array{Int}, ones(size(fire_dists)))

# initialize number of periods to travel from base c to fire g (NUM_CREWS-by-NUM_FIRES)
tau_base_to_fire = convert(Array{Int}, ones((size(base_fire_dists))))

# read intial crew statuses (location, period by which they must rest)
# (-1 in current_fire means crew is currently at base)
# (rested_periods is the amount of time crew has been at base, relevant for completing rest)
crew_starts = CSV.read(in_path * "/sample_crew_starts.csv", DataFrame)
println(size(crew_starts)[1])
rest_by = crew_starts[!, "rest_by"]
current_fire = crew_starts[!, "current_fire"]
rested_periods = crew_starts[!, "rested_periods"];

10


## Helper functions for full formulation

In [5]:
struct ArcData
    
    ff_ix::Vector
    fr_ix::Vector
    rf_ix::Vector
    rr_ix::Vector
    
    ff_ix_arr::Vector
    fr_ix_arr::Vector
    rf_ix_arr::Vector
    rr_ix_arr::Vector
    
    from_start_ff::Vector
    from_start_fr::Vector
    from_start_rf::Vector
    from_start_rr::Vector

end

In [6]:
struct NetworkConstraintData
    
    # fire flow data
    ff_out::Array{Vector{NTuple},4}
    fr_out::Array{Vector{NTuple},4}
    ff_in::Array{Vector{NTuple},4}
    rf_in::Array{Vector{NTuple},4}
    
    rf_out::Array{Vector{NTuple},3}
    rr_out::Array{Vector{NTuple},3}
    fr_in::Array{Vector{NTuple},3}
    rr_in::Array{Vector{NTuple},3}
    
    ff_link::Array{Vector{NTuple},2}
    rf_link::Array{Vector{NTuple},2}
    
end 

In [7]:
struct FireProgressionData
    
    start_perims::Vector
    progressions::Matrix

end

In [8]:
function define_arc_sets(tau, tau_base_to_fire, current_fire, rest_by, rested_periods)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    ## generate fire_to_fire index set

    # arcs for time-space network for non-rested crews
    non_rested_ff = [(c, f_from, f_to, t_from, t_from + tau[f_to, f_from], 0)
                      for c=1:C, f_from=1:G, f_to=1:G, t_from=1:T
                      if t_from <= rest_by[c]]

    # arcs for time-space network for rested crews
    rested_ff = [(c, f_from, f_to, t_from, t_from + tau[f_to, f_from], 1)
                   for c=1:C, f_from=1:G, f_to=1:G, t_from=1:T
                   if 1==1]

    # special arcs for first travel from initial fires
    from_start_ff = [(c, current_fire[c], f_to, 0, tau[f_to, current_fire[c]], 0)
                      for c=1:C, f_to=1:G
                      if current_fire[c] != -1]

    # concat
    ff_ix = vcat(non_rested_ff, rested_ff)
    ff_ix = vcat(from_start_ff, ff_ix)

    ## generate fire_to_rest index set

    # arcs for time-space network for non-rested crews
    non_rested_fr = [(c, f_from, t_from, t_from + tau_base_to_fire[c, f_from], 0)
                      for c=1:C, f_from=1:G, t_from=1:T
                      if t_from <= rest_by[c]]

    # arcs for time-space network for rested crews
    rested_fr = [(c, f_from, t_from, t_from + tau_base_to_fire[c, f_from], 1)
                   for c=1:C, f_from=1:G, t_from=1:T
                   if 1==1]

    # special arcs for first travel from initial fires
    from_start_fr = [(c, current_fire[c], 0, tau_base_to_fire[c, current_fire[c]], 0)
                      for c=1:C
                      if current_fire[c] != -1]

    # concat
    fr_ix = vcat(non_rested_fr, rested_fr)
    fr_ix = vcat(from_start_fr, fr_ix)

    ## generate rest_to_fire index set

    # arcs for time-space network when leaving base without rest
    non_rested_rf = [(c, f_to, t_from, t_from + tau_base_to_fire[c, f_to], 0)
                       for c=1:C, f_to=1:G, t_from=1:T
                       if t_from <= rest_by[c]]

    # arcs for time-space network when going non-rest -> rest
    rested_rf = [(c, f_to, t_from, t_from + tau_base_to_fire[c, f_to], 1)
                   for c=1:C, f_to=1:G, t_from=1:T
                   if 1==1]

    # special arcs for first travel from initial rest
    from_start_rf = [(c, f_to, 0, tau_base_to_fire[c, f_to], 0)
                      for c=1:C, f_to=1:G
                      if current_fire[c] == -1]

    # concat
    rf_ix = vcat(non_rested_rf, rested_rf)
    rf_ix = vcat(from_start_rf, rf_ix)

    ## generate rest_to_rest index set
    ## (crew, time_from, time_to, rested_From, rested_to)


    non_to_non_rested_rr = [(c, t_from, t_from + 1, 0, 0)
                               for c=1:C, t_from=1:T
                               if t_from <= rest_by[c]]

    non_to_yes_rested_rr = [(c, t_from, t_from + BREAK_LENGTH, 0, 1)
                               for c=1:C, t_from=1:T
                               if t_from <= rest_by[c]]

    yes_rested_rr = [(c, t_from, t_from + 1, 1, 1)
                       for c=1:C, t_from=1:T
                        if 1==1]

    from_start_to_non_rested_rr = [(c, 0, 1, 0, 0) for c=1:C if current_fire[c] == -1]
    from_start_to_yes_rested_rr = [(c, 0, BREAK_LENGTH - rested_periods[c], 0, 1) 
                                      for c=1:C 
                                      if (current_fire[c] == -1) & (rested_periods[c] != -1)]


    rr_ix = vcat(non_to_non_rested_rr, non_to_yes_rested_rr)
    rr_ix = vcat(rr_ix, yes_rested_rr)
    rr_ix = vcat(from_start_to_non_rested_rr, rr_ix)
    rr_ix = vcat(from_start_to_yes_rested_rr, rr_ix)

    ff_ix_arr = [[ix for ix in ff_ix if ix[1] == c] for c=1:C]
    fr_ix_arr = [[ix for ix in fr_ix if ix[1] == c] for c=1:C]
    rf_ix_arr = [[ix for ix in rf_ix if ix[1] == c] for c=1:C]
    rr_ix_arr = [[ix for ix in rr_ix if ix[1] == c] for c=1:C]
    
    return ArcData(ff_ix, fr_ix, rf_ix, rr_ix, ff_ix_arr, fr_ix_arr, rf_ix_arr, rr_ix_arr,
                   from_start_ff, from_start_fr, from_start_rf, 
                   vcat(from_start_to_yes_rested_rr, from_start_to_non_rested_rr))
end

define_arc_sets (generic function with 1 method)

In [9]:
function define_network_constraint_data(arc_data)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # build fire flow constraint data
    ff_outflows = Array{Vector{NTuple},4}(undef, C, G, T, 2)
    fr_outflows = Array{Vector{NTuple},4}(undef, C, G, T, 2)
    ff_inflows = Array{Vector{NTuple},4}(undef, C, G, T, 2)
    rf_inflows = Array{Vector{NTuple},4}(undef, C, G, T, 2)

    for c=1:C
        for g=1:G
            for t=1:T
                for rest=0:1

                    S = [key for key in arc_data.ff_ix_arr[c] if (key[2] == g) & (key[4] == t) & (key[6] == rest)]
                    ff_outflows[c, g, t, rest+1] = copy(S)

                    S = [key for key in arc_data.fr_ix_arr[c] if (key[2] == g) & (key[3] == t) & (key[5] == rest)]
                    fr_outflows[c, g, t, rest+1] = copy(S)

                    S = [key for key in arc_data.ff_ix_arr[c] if (key[3] == g) & (key[5] == t) & (key[6] == rest)]
                    ff_inflows[c, g, t, rest+1] = copy(S)

                    S = [key for key in arc_data.rf_ix_arr[c] if (key[2] == g) & (key[4] == t) & (key[5] == rest)]
                    rf_inflows[c, g, t, rest+1] = copy(S)
                end
            end
        end
    end
    
    
    # build rest flow constraint data
    rf_outflows = Array{Vector{NTuple},3}(undef, C, T, 2)
    rr_outflows = Array{Vector{NTuple},3}(undef, C, T, 2)
    fr_inflows = Array{Vector{NTuple},3}(undef, C, T, 2)
    rr_inflows = Array{Vector{NTuple},3}(undef, C, T, 2)

    for c=1:C
        for t=1:T
            for rest=0:1

                S = [key for key in arc_data.rf_ix_arr[c] if (key[3] == t) & (key[5] == rest)]
                rf_outflows[c, t, rest+1] = copy(S)

                S = [key for key in arc_data.rr_ix_arr[c] if (key[2] == t) & (key[4] == rest)]
                rr_outflows[c, t, rest+1] = copy(S)

                S = [key for key in arc_data.fr_ix_arr[c] if (key[4] == t) &  (key[5] == rest)]
                fr_inflows[c, t, rest+1] = copy(S)

                S = [key for key in arc_data.rr_ix_arr[c] if (key[3] == t) & (key[5] == rest)]
                rr_inflows[c, t, rest+1] = copy(S)

            end
        end
    end
    
    # bulid linking constraint data
    ff_linking = Array{Vector{NTuple},2}(undef, G, T)
    rf_linking = Array{Vector{NTuple},2}(undef, G, T)

    for g=1:G
        for t=1:T

            S = [key for key in arc_data.ff_ix if (key[3] == g) & (key[5] == t)]
            ff_linking[g, t] = copy(S)

            S = [key for key in arc_data.rf_ix if (key[2] == g) & (key[4] == t)]
            rf_linking[g, t] = copy(S)

        end
    end
    
    return NetworkConstraintData(ff_outflows, fr_outflows, ff_inflows, rf_inflows, rf_outflows, rr_outflows,
                                 fr_inflows, rr_inflows, ff_linking, rf_linking)
end

define_network_constraint_data (generic function with 1 method)

In [10]:
function full_formulation(integer_routes, arc_data, fire_data, constraint_data)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # fire suppression plan section
    @variable(m, p[g=1:G, t=1:T+1] >= 0)
    @variable(m, l[g=1:G, t=1:T] >= 0)
    @constraint(m, perim_growth[g=1:G, t=1:T], p[g, t+1] >= fire_data.progressions[g, t] * 
                                                           (p[g, t] - l[g, t] / 2) - l[g, t] / 2)
    @constraint(m, perim_start[g=1:G], p[g, 1] == fire_data.start_perims[g])

    # routing plan section
    if integer_routes
        @variable(m, ff[arc_data.ff_ix] >= 0, Int)
        @variable(m, fr[arc_data.fr_ix] >= 0, Int)
        @variable(m, rf[arc_data.rf_ix] >= 0, Int)
        @variable(m, rr[arc_data.rr_ix] >= 0, Int)
    else
        @variable(m, ff[arc_data.ff_ix] >= 0)
        @variable(m, fr[arc_data.fr_ix] >= 0)
        @variable(m, rf[arc_data.rf_ix] >= 0)
        @variable(m, rr[arc_data.rr_ix] >= 0)
    end


    
    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=0:1],

            sum(ff[ix] for ix in constraint_data.ff_out[c, g, t, rest+1]) + 
            sum(fr[ix] for ix in constraint_data.fr_out[c, g, t, rest+1]) ==
            sum(ff[ix] for ix in constraint_data.ff_in[c, g, t, rest+1]) + 
            sum(rf[ix] for ix in constraint_data.rf_in[c, g, t, rest+1])
    
    )

    @constraint(m, rest_flow[c=1:C, t=1:T, rest=0:1],

            sum(rf[ix] for ix in constraint_data.rf_out[c, t, rest+1]) + 
            sum(rr[ix] for ix in constraint_data.rr_out[c, t, rest+1]) ==
            sum(fr[ix] for ix in constraint_data.fr_in[c, t, rest+1]) + 
            sum(rr[ix] for ix in constraint_data.rr_in[c, t, rest+1])
    
    )

    @constraint(m, linking[g=1:G, t=1:T],

         sum(ff[key] for key in constraint_data.ff_link[g, t]) + 
         sum(rf[key] for key in constraint_data.rf_link[g, t]) >= 
         l[g, t] / LINE_PER_CREW
    )

    # build start constraint
    @constraint(m, start[c=1:C], 

        sum(ff[key] for key in arc_data.from_start_ff if key[1] == c) + 
        sum(rf[key] for key in arc_data.from_start_rf if key[1] == c) + 
        sum(fr[key] for key in arc_data.from_start_fr if key[1] == c) + 
        sum(rr[key] for key in arc_data.from_start_rr if key[1] == c) == 1
               )

    @objective(m, Min, 
        BETA * (sum(p) - sum(p[1:G, 1])/2 - sum(p[1:G, T+1])/2) + 
        sum(ff[key] * (ALPHA + fire_dists[key[2], key[3]]) for key in arc_data.ff_ix) +
        sum(fr[key] * (base_fire_dists[key[1], key[2]]) for key in arc_data.fr_ix) + 
        sum(rf[key] * (ALPHA + base_fire_dists[key[1], key[2]]) for key in arc_data.rf_ix))
    
    return m, p, l, ff, fr, rf, rr
    
end

full_formulation (generic function with 1 method)

## Subproblems

### Initialize subproblems for each crew and fire

In [11]:
function init_route_subproblem(arc_data, crew)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # routing plan section
    @variable(m, ff[arc_data.ff_ix_arr[crew]] >= 0)
    @variable(m, fr[arc_data.fr_ix_arr[crew]] >= 0)
    @variable(m, rf[arc_data.rf_ix_arr[crew]] >= 0)
    @variable(m, rr[arc_data.rr_ix_arr[crew]] >= 0)


    @constraint(m, fire_flow[g=1:G, t=1:T, rest=0:1],

                # outflow
                sum(ff[key] for key in arc_data.ff_ix_arr[crew]
                        if (key[2] == g) & (key[4] == t) & (key[6] == rest)
                    ) +    

                sum(fr[key] for key in arc_data.fr_ix_arr[crew]
                            if (key[2] == g) & (key[3] == t) & (key[5] == rest)
                    ) 

        ==
                # inflow
                sum(ff[key] for key in arc_data.ff_ix_arr[crew]
                            if (key[3] == g) & (key[5] == t) & (key[6] == rest)
                    ) +

                sum(rf[key] for key in arc_data.rf_ix_arr[crew]
                            if (key[2] == g) & (key[4] == t) & (key[5] == rest)
                    ) 

                )   

    @constraint(m, rest_flow[t=1:T, rest=0:1], 

                # outflow
                sum(rf[key] for key in arc_data.rf_ix_arr[crew]
                            if (key[3] == t) & (key[5] == rest)
                    ) +

                sum(rr[key] for key in arc_data.rr_ix_arr[crew]
                            if (key[2] == t) & (key[4] == rest)
                    )

                ==       

                # inflow
                sum(fr[key] for key in arc_data.fr_ix_arr[crew]
                            if (key[4] == t) &  (key[5] == rest)
                    ) +
                sum(rr[key] for key in arc_data.rr_ix_arr[crew]
                            if (key[3] == t) & (key[5] == rest)
                    )
               )

    @constraint(m, start, 
        sum(ff[key] for key in arc_data.from_start_ff if key[1] == crew) + 
        sum(rf[key] for key in arc_data.from_start_rf if key[1] == crew) + 
        sum(fr[key] for key in arc_data.from_start_fr if key[1] == crew) + 
        sum(rr[key] for key in arc_data.from_start_rr if key[1] == crew) == 1)

    return Dict("m" => m, "ff" => ff, "fr" => fr, "rf" => rf, "rr" => rr)
end

init_route_subproblem (generic function with 1 method)

In [12]:
function init_route_subproblem(arc_data, constraint_data, crew)
    
    # shorten some global variable names
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # routing plan section
    @variable(m, ff[arc_data.ff_ix_arr[crew]] >= 0)
    @variable(m, fr[arc_data.fr_ix_arr[crew]] >= 0)
    @variable(m, rf[arc_data.rf_ix_arr[crew]] >= 0)
    @variable(m, rr[arc_data.rr_ix_arr[crew]] >= 0)

    
    @constraint(m, fire_flow[g=1:G, t=1:T, rest=0:1],

            sum(ff[ix] for ix in constraint_data.ff_out[crew, g, t, rest+1]) + 
            sum(fr[ix] for ix in constraint_data.fr_out[crew, g, t, rest+1]) ==
            sum(ff[ix] for ix in constraint_data.ff_in[crew, g, t, rest+1]) + 
            sum(rf[ix] for ix in constraint_data.rf_in[crew, g, t, rest+1])
    
    )

    @constraint(m, rest_flow[t=1:T, rest=0:1],

            sum(rf[ix] for ix in constraint_data.rf_out[crew, t, rest+1]) + 
            sum(rr[ix] for ix in constraint_data.rr_out[crew, t, rest+1]) ==
            sum(fr[ix] for ix in constraint_data.fr_in[crew, t, rest+1]) + 
            sum(rr[ix] for ix in constraint_data.rr_in[crew, t, rest+1])
    
    )

    @constraint(m, start, 
        sum(ff[key] for key in arc_data.from_start_ff if key[1] == crew) + 
        sum(rf[key] for key in arc_data.from_start_rf if key[1] == crew) + 
        sum(fr[key] for key in arc_data.from_start_fr if key[1] == crew) + 
        sum(rr[key] for key in arc_data.from_start_rr if key[1] == crew) == 1)

    return Dict("m" => m, "ff" => ff, "fr" => fr, "rf" => rf, "rr" => rr)
end

init_route_subproblem (generic function with 2 methods)

In [13]:
function init_suppression_plan_subproblem(fire_data, fire)
    
    T = NUM_TIME_PERIODS
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # fire suppression plan section
    @variable(m, p[t=1:T+1] >= 0)
    @variable(m, l[t=1:T] >= 0)
    @variable(m, NUM_CREWS >= d[t=1:T] >= 0, Int)
    @constraint(m, suppression_per_crew[t=1:T], l[t] <= d[t] * LINE_PER_CREW)
    @constraint(m, perim_growth[t=1:T], p[t+1] >= fire_data.progressions[fire, t] * (p[t] - l[t] / 2) - l[t] / 2)
    @constraint(m, perim_start, p[1] == fire_data.start_perims[fire])
    
    @objective(m, Min, BETA * (sum(p) - p[1]/2 - p[T+1]/2))
    
    return Dict("m" => m, "p" => p, "d" => d)
end

init_suppression_plan_subproblem (generic function with 1 method)

In [14]:
function route_subproblem(arc_data, crew, model_dict, rho)
    
    T = NUM_TIME_PERIODS
    
    m = model_dict["m"]
    ff = model_dict["ff"]
    fr = model_dict["fr"]
    rf = model_dict["rf"]
    
    @objective(m, Min, 
    sum(ff[key] * (ALPHA + fire_dists[key[2], key[3]]) for key in arc_data.ff_ix_arr[crew]) +
    sum(fr[key] * (base_fire_dists[key[1], key[2]]) for key in arc_data.fr_ix_arr[crew]) + 
    sum(rf[key] * (ALPHA + base_fire_dists[key[1], key[2]]) for key in arc_data.rf_ix_arr[crew]) -

    sum(ff[key] * rho[key[3], key[5]] for key in arc_data.ff_ix_arr[crew] if key[5] <= T) -
    sum(rf[key] * rho[key[2], key[4]] for key in arc_data.rf_ix_arr[crew] if key[4] <= T)
    )

    optimize!(m)
    
    fires_fought = vcat([(ix[1], ix[3], ix[5]) for ix in arc_data.ff_ix_arr[crew] 
                                               if (value(ff[ix]) > 0.99)],
                        [(ix[1], ix[2], ix[4]) for ix in arc_data.rf_ix_arr[crew] 
                                               if (value(rf[ix]) > 0.99)])
        
    route_cost = 
    sum(value(ff[key]) * (ALPHA + fire_dists[key[2], key[3]]) 
                                            for key in arc_data.ff_ix_arr[crew]) +
    sum(value(fr[key]) * (base_fire_dists[key[1], key[2]]) 
                                            for key in arc_data.fr_ix_arr[crew]) + 
    sum(value(rf[key]) * (ALPHA + base_fire_dists[key[1], key[2]]) 
                                            for key in arc_data.rf_ix_arr[crew])
    
    objective_value(m), sort(fires_fought), route_cost
    
end

route_subproblem (generic function with 1 method)

In [15]:
function suppression_plan_subproblem(g, model_dict, rho, second_objective_eps)
    
    T = NUM_TIME_PERIODS
    
    model = model_dict["m"]
    p = model_dict["p"]
    d = model_dict["d"]
    
    @objective(model, Min, BETA * (sum(p) - p[1]/2 - p[T+1]/2) + 
                       sum(d[t] * (rho[g, t] + second_objective_eps) for t=1:T)
    )
        
    optimize!(model)
    
    relative_cost = objective_value(model) - sum(value(d[t]) * second_objective_eps for t=1:T)
    pure_cost = objective_value(model) - 
                sum(value(d[t]) * (rho[g, t] + second_objective_eps) for t=1:T) 
        
    return round.(value.(d)), pure_cost, relative_cost
    
end

suppression_plan_subproblem (generic function with 1 method)

## Helper functions for warm start

assumes we suppress each fire more than 0, first day not full, can revisit

In [16]:
function suppression_plan_perturbations(start_plan, count)
    
    # get the indices we may perturb, chosen to be anything at most one index away
    # from a time period when suppression was >0 for the start_plan
    ixs = [i for i in 1:NUM_TIME_PERIODS if start_plan[i] > 0]
    ixs_1 = [i+1 for i in 1:NUM_TIME_PERIODS-1 if start_plan[i] > 0]
    ixs_2 = [i-1 for i in 2:NUM_TIME_PERIODS if start_plan[i] > 0]
    ixs_to_perturb = sort(unique(vcat(ixs, ixs_1, ixs_2)))
    
    start_plan_copy = copy(start_plan)
    
    # hack if no fire suppression
    if length(ixs) == 0
        start_plan_copy[1] += 1
        ixs_to_perturb = 1:NUM_TIME_PERIODS
        ixs = [1]
    end
    
    found = []
    
    # perturbing the total number of crews by 0, -1, or 1
    for perturb in [0, 1, -1]
        
        # make sure we don't explode
        curr_length = length(found)
        
        # push a new possible plan
        new_plan = copy(start_plan_copy)
        new_plan[ixs[1]] = new_plan[ixs[1]] + perturb
        push!(found, copy(new_plan))
        
        # find all ways to perturb the valid indices while saying within
        # prescribed crew bounds and keeping total crews unchanged
        for current_arr in found
            if length(found) < curr_length + 500
                nonzero = [ix for ix in ixs_to_perturb if current_arr[ix] > 0]
                nonfull = [ix for ix in ixs_to_perturb if current_arr[ix] < NUM_CREWS]
                for ix in nonzero
                    for ix2 in nonfull
                        if ix2 != ix
                            next_arr = copy(current_arr)
                            next_arr[ix] -= 1
                            next_arr[ix2] += 1
                            if !(next_arr in found)
                                push!(found, copy(next_arr))
                            end
                        end
                    end
                end
            end
        end
    end
    
    # get the closest plans to the original, using L1 norm
    ixs_to_keep = sortperm([sum(abs.(i - start_plan)) for i in found])[1:min(count, length(found))]
    
    # add in dummy plan, sort by total crews used, return
    out = found[ixs_to_keep]
    push!(out, [0 for i in 1:NUM_TIME_PERIODS])
    ixs = sortperm([sum(i) for i in out])
    
    return out[ixs]
end 

suppression_plan_perturbations (generic function with 1 method)

In [17]:
function fire_plan_cost(start_perim, line_per_day, progression)
    
    perim = start_perim
    area = 0
    new_perim = 0
    for i in 1:length(progression)
        new_perim = (perim - line_per_day[i]/2) * progression[i] - line_per_day[i]/2
        new_perim = max(new_perim, 0)
        area = area + perim/2 + new_perim/2
        perim = new_perim
    end
    area
end

fire_plan_cost (generic function with 1 method)

In [18]:
function update_suppression_plan_data(fire_data, B, supp_costs, fire, num_plans, plan)
    
    B[num_plans + 1, fire, :] = plan
    supp_costs[num_plans + 1, fire] = BETA * fire_plan_cost(fire_data.start_perims[fire], 
                                                            plan * LINE_PER_CREW, 
                                                            fire_data.progressions[fire, :])
    
    return B, supp_costs, num_plans + 1
end

update_suppression_plan_data (generic function with 1 method)

In [19]:
function init_routes_from_plan_formulation(arc_data)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)


    @variable(m, ff[arc_data.ff_ix] >= 0)
    @variable(m, fr[arc_data.fr_ix] >= 0)
    @variable(m, rf[arc_data.rf_ix] >= 0)
    @variable(m, rr[arc_data.rr_ix] >= 0)



    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=0:1],

                # outflow
                sum(ff[key] for key in arc_data.ff_ix_arr[c]
                        if (key[2] == g) & (key[4] == t) & (key[6] == rest)
                    ) +    

                sum(fr[key] for key in arc_data.fr_ix_arr[c]
                            if (key[2] == g) & (key[3] == t) & (key[5] == rest)
                    ) 

        ==
                # inflow
                sum(ff[key] for key in arc_data.ff_ix_arr[c]
                            if (key[3] == g) & (key[5] == t) & (key[6] == rest)
                    ) +

                sum(rf[key] for key in arc_data.rf_ix_arr[c]
                            if (key[2] == g) & (key[4] == t) & (key[5] == rest)
                    ) 

                )   

    @constraint(m, rest_flow[c=1:C, t=1:T, rest=0:1], 

                # outflow
                sum(rf[key] for key in arc_data.rf_ix_arr[c]
                            if (key[3] == t) & (key[5] == rest)
                    ) +

                sum(rr[key] for key in arc_data.rr_ix_arr[c]
                            if (key[2] == t) & (key[4] == rest)
                    )

                ==       

                # inflow
                sum(fr[key] for key in arc_data.fr_ix_arr[c]
                            if (key[4] == t) &  (key[5] == rest)
                    ) +
                sum(rr[key] for key in arc_data.rr_ix_arr[c]
                            if (key[3] == t) & (key[5] == rest)
                    )
               )

    @constraint(m, start[c=1:C], 

        sum(ff[key] for key in arc_data.from_start_ff if key[1] == c) + 
        sum(rf[key] for key in arc_data.from_start_rf if key[1] == c) + 
        sum(fr[key] for key in arc_data.from_start_fr if key[1] == c) + 
        sum(rr[key] for key in arc_data.from_start_rr if key[1] == c) == 1
               )
    
    @constraint(m, linking[g=1:G, t=1:T], 0 == 0)
    
    @objective(m, Min, 
    sum(ff[key] * (ALPHA + fire_dists[key[2], key[3]]) for key in arc_data.ff_ix) +
    sum(fr[key] * (base_fire_dists[key[1], key[2]]) for key in arc_data.fr_ix) + 
    sum(rf[key] * (ALPHA + base_fire_dists[key[1], key[2]]) for key in arc_data.rf_ix))

    return Dict("m" => m, "ff" => ff, "rf" => rf, "fr" => fr, "linking_constraint" => linking)

end

init_routes_from_plan_formulation (generic function with 1 method)

In [20]:
function init_routes_from_plan_formulation(arc_data, constraint_data)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)


    @variable(m, ff[arc_data.ff_ix] >= 0)
    @variable(m, fr[arc_data.fr_ix] >= 0)
    @variable(m, rf[arc_data.rf_ix] >= 0)
    @variable(m, rr[arc_data.rr_ix] >= 0)



    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=0:1],

            sum(ff[ix] for ix in constraint_data.ff_out[c, g, t, rest+1]) + 
            sum(fr[ix] for ix in constraint_data.fr_out[c, g, t, rest+1]) ==
            sum(ff[ix] for ix in constraint_data.ff_in[c, g, t, rest+1]) + 
            sum(rf[ix] for ix in constraint_data.rf_in[c, g, t, rest+1])
    
    )

    @constraint(m, rest_flow[c=1:C, t=1:T, rest=0:1],

            sum(rf[ix] for ix in constraint_data.rf_out[c, t, rest+1]) + 
            sum(rr[ix] for ix in constraint_data.rr_out[c, t, rest+1]) ==
            sum(fr[ix] for ix in constraint_data.fr_in[c, t, rest+1]) + 
            sum(rr[ix] for ix in constraint_data.rr_in[c, t, rest+1])
    
    )

    @constraint(m, start[c=1:C], 

        sum(ff[key] for key in arc_data.from_start_ff if key[1] == c) + 
        sum(rf[key] for key in arc_data.from_start_rf if key[1] == c) + 
        sum(fr[key] for key in arc_data.from_start_fr if key[1] == c) + 
        sum(rr[key] for key in arc_data.from_start_rr if key[1] == c) == 1
               )
    
    @constraint(m, linking[g=1:G, t=1:T], 0 == 0)
    
    @objective(m, Min, 
    sum(ff[key] * (ALPHA + fire_dists[key[2], key[3]]) for key in arc_data.ff_ix) +
    sum(fr[key] * (base_fire_dists[key[1], key[2]]) for key in arc_data.fr_ix) + 
    sum(rf[key] * (ALPHA + base_fire_dists[key[1], key[2]]) for key in arc_data.rf_ix))

    return Dict("m" => m, "ff" => ff, "rf" => rf, "fr" => fr, "linking_constraint" => linking)

end

init_routes_from_plan_formulation (generic function with 2 methods)

In [21]:
function get_routes_from_demands(arc_data, constraint_data, model_dict, demands)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    m = model_dict["m"]
    ff = model_dict["ff"]
    rf = model_dict["rf"]
    fr = model_dict["fr"]
    old_linking = model_dict["linking_constraint"]

    
    # overwrite demand constraint
    unregister(m, :linking)
    [delete(m, i) for i in old_linking]
    

    @constraint(m, linking[g=1:G, t=1:T],

        sum(ff[key] for key in constraint_data.ff_link[g, t]) + 
        sum(rf[key] for key in constraint_data.rf_link[g, t])
        >= demands[g, t]
    )
    model_dict["linking_constraint"] = linking
    
    # optimize model
    optimize!(m)
    
    # extract routes
    opt = false
    fires_fought = [(0)]
    route_costs = [1]
    if termination_status(m) == MOI.OPTIMAL
        opt = true
        fires_fought = vcat([(ix[1], ix[3], ix[5]) for ix in arc_data.ff_ix if (value(ff[ix]) > 0.99)],
                        [(ix[1], ix[2], ix[4]) for ix in arc_data.rf_ix if (value(rf[ix]) > 0.99)])
        
        route_costs = 
        [sum(value(ff[key]) * (ALPHA + fire_dists[key[2], key[3]]) for key in arc_data.ff_ix_arr[crew]) +
        sum(value(fr[key]) * (base_fire_dists[key[1], key[2]]) for key in arc_data.fr_ix_arr[crew]) + 
        sum(value(rf[key]) * (ALPHA + base_fire_dists[key[1], key[2]]) for key in arc_data.rf_ix_arr[crew])
            for crew = 1:C]
        
    else
        opt, fires_fought, route_costs = get_routes_from_demands(arc_data, constraint_data, model_dict, max.(demands .- 1, 0))
    end
    opt, fires_fought, route_costs
    
end

get_routes_from_demands (generic function with 1 method)

In [22]:
function update_crew_route_data(A, route_costs, crew, num_routes, daily_assignments, cost)
    
    matrix_route = zeros(NUM_FIRES, NUM_TIME_PERIODS)
    for assignment in daily_assignments
        if assignment[3] <= NUM_TIME_PERIODS + 0.001
            matrix_route[assignment[2], assignment[3]] = 1
        end
    end
    A[num_routes + 1, crew, :, :] = matrix_route
    route_costs[num_routes + 1, crew] = cost
    
    return A, route_costs
end

update_crew_route_data (generic function with 1 method)

In [23]:
function create_warm_start_suppression_plans(fire_data, l, count)
        
    B = zeros((1000, NUM_FIRES, NUM_TIME_PERIODS))
    num_plans_per_fire = convert.(Int, zeros(NUM_FIRES))
    suppression_costs = zeros((1000, NUM_FIRES))

    assignments = round.(value.(l)/LINE_PER_CREW)[:, 1:NUM_TIME_PERIODS]
    for fire in 1:NUM_FIRES
        plans = suppression_plan_perturbations(assignments[fire, :], count)
        for plan in plans
            B, suppression_costs, _ = update_suppression_plan_data(fire_data, B, suppression_costs, fire,
                                                                   num_plans_per_fire[fire], plan)
            num_plans_per_fire[fire] += 1
        end
    end
    return B, suppression_costs, num_plans_per_fire
end

create_warm_start_suppression_plans (generic function with 1 method)

In [24]:
function heuristic_route(arc_data, model_dict, crew, ff_val, fr_val, rf_val, rr_val)
    
    T = NUM_TIME_PERIODS

    m = model_dict["m"]
    ff = model_dict["ff"]
    fr = model_dict["fr"]
    rf = model_dict["rf"]
    rr = model_dict["rr"]

    @objective(m, Max,
    sum(ff[key] * ff_val[key] for key in arc_data.ff_ix_arr[crew]) +
    sum(fr[key] * fr_val[key] for key in arc_data.fr_ix_arr[crew]) + 
    sum(rf[key] * rf_val[key] for key in arc_data.rf_ix_arr[crew]) + 
    sum(rr[key] * rr_val[key] for key in arc_data.rr_ix_arr[crew])
        )

    optimize!(m)

    fires_fought = vcat([(ix[1], ix[3], ix[5]) for ix in arc_data.ff_ix_arr[crew] 
                                               if (value(ff[ix]) > 0.99)],
                        [(ix[1], ix[2], ix[4]) for ix in arc_data.rf_ix_arr[crew] 
                                               if (value(rf[ix]) > 0.99)])

    route_cost = 
    sum(value(ff[key]) * (ALPHA + fire_dists[key[2], key[3]]) 
                                            for key in arc_data.ff_ix_arr[crew]) +
    sum(value(fr[key]) * (base_fire_dists[key[1], key[2]]) 
                                            for key in arc_data.fr_ix_arr[crew]) + 
    sum(value(rf[key]) * (ALPHA + base_fire_dists[key[1], key[2]]) 
                                            for key in arc_data.rf_ix_arr[crew])

    sort(fires_fought), route_cost
    
end

heuristic_route (generic function with 1 method)

In [25]:
function fast_create_warm_start_crew_routes(arc_data, constraint_data, dummy_fire_plan, sp_models, 
                                            ff_val, fr_val, rf_val, rr_val)
    t = 0
    u = 0
    v = 0
    t = @elapsed model_dict = init_routes_from_plan_formulation(arc_data, constraint_data)
    A = zeros(1000, NUM_CREWS, NUM_FIRES, NUM_TIME_PERIODS)
    num_routes_per_crew = convert.(Int, zeros(NUM_CREWS))
    crew_route_costs = zeros((1000, NUM_CREWS))
    
    assignments_by_crew = [[] for i in 1:NUM_CREWS]

    # get routes corresponding to dummy plan
    fires_fought = false
    
    u += @elapsed _, fires_fought, route_costs = get_routes_from_demands(arc_data, constraint_data, model_dict, dummy_fire_plan)

    # for each crew
    for crew in 1:NUM_CREWS

        # get their assignments and the route cost
        daily_assignments = sort([j for j in fires_fought if j[1] == crew])
        cost = route_costs[crew]

        # if we have not already seen this route, add it and make appropriate updates
        if !(daily_assignments in assignments_by_crew[crew])

            A, crew_route_costs = update_crew_route_data(A, crew_route_costs, crew, num_routes_per_crew[crew], 
                                                    daily_assignments, cost)

            num_routes_per_crew[crew] += 1
        end
    end
    
    
    # get routes corresponding to best warm start
    fires_fought = false
    
    # for each crew
    for crew in 1:NUM_CREWS
        
        v += @elapsed daily_assignments, cost = heuristic_route(arc_data, sp_models[crew], crew, ff_val, fr_val, rf_val, rr_val)

        # if we have not already seen this route, add it and make appropriate updates
        if !(daily_assignments in assignments_by_crew[crew])

            A, crew_route_costs = update_crew_route_data(A, crew_route_costs, crew, num_routes_per_crew[crew], 
                                                    daily_assignments, cost)

            num_routes_per_crew[crew] += 1
        end
    end
    
    

    println(t)
    println(u)
    println(v)
    return A, crew_route_costs, num_routes_per_crew
end

fast_create_warm_start_crew_routes (generic function with 1 method)

In [26]:
function create_warm_start_crew_routes(arc_data, constraint_data, plans, max_per_crew)
    
    model_dict = init_routes_from_plan_formulation(arc_data, constraint_data)
    A = zeros(1000, NUM_CREWS, NUM_FIRES, NUM_TIME_PERIODS)
    num_routes_per_crew = convert.(Int, zeros(NUM_CREWS))
    crew_route_costs = zeros((1000, NUM_CREWS))
    
    num_plans = size(plans)[1]
    
    assignments_by_crew = [[] for i in 1:NUM_CREWS]
    for i = 1:num_plans
        
        if minimum(num_routes_per_crew) < max_per_crew
        
            opt, fires_fought, route_costs = get_routes_from_demands(arc_data, constraint_data, model_dict, plans[i, :, :])

            # should always be here
            if opt

                # for each crew
                for crew in 1:NUM_CREWS

                    # get their assignments and the route cost
                    daily_assignments = sort([j for j in fires_fought if j[1] == crew])
                    cost = route_costs[crew]

                    # if we have not already seen this route, add it and make appropraite updates
                    if !(daily_assignments in assignments_by_crew[crew]) & (num_routes_per_crew[crew] < max_per_crew)

                        A, crew_route_costs = update_crew_route_data(A, crew_route_costs, crew, num_routes_per_crew[crew], 
                                                                daily_assignments, cost)

                        num_routes_per_crew[crew] += 1
                    end
                end
            end
        end
    end
    return A, crew_route_costs, num_routes_per_crew
end

create_warm_start_crew_routes (generic function with 1 method)

## CG functions

In [27]:
function create_master_problem_model()
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)
    return 0
    return m
end

create_master_problem_model (generic function with 1 method)

In [28]:
function run_master_problem_step(crew_route_count_dict, fire_plan_count_dict, route_costs, fire_costs, A, B)
    
    # make variable index sets
    fire_plan_ix = [(i, g) for g=1:NUM_FIRES for i=1:fire_plan_count_dict[g]]
    route_plan_ix = [(i, c) for c=1:NUM_CREWS for i=1:crew_route_count_dict[c]]
    
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    # formulate "disappointingly small" formulation
    @variable(model, suppression_plans[fire_plan_ix] >= 0)
    @variable(model, crew_routes[route_plan_ix] >= 0)

    @constraint(model, plan_per_fire[g=1:NUM_FIRES], 
                    sum(suppression_plans[ix] for ix in fire_plan_ix if ix[2] == g) >= 1)
    @constraint(model, route_per_crew[c=1:NUM_CREWS], 
                    sum(crew_routes[ix] for ix in route_plan_ix if ix[2] == c) == 1)
    @constraint(model, cover_plans[g=1:NUM_FIRES, t=1:NUM_TIME_PERIODS],
                    sum(crew_routes[ix] * A[ix[1], ix[2], g, t] for ix in route_plan_ix) >=
                    sum(suppression_plans[ix] * B[ix[1], g, t] for ix in fire_plan_ix if ix[2] == g)
               )
    @objective(model, Min, 
                  sum(crew_routes[ix] * route_costs[ix[1], ix[2]] for ix in route_plan_ix) + 
                  sum(suppression_plans[ix] * fire_costs[ix[1], ix[2]] for ix in fire_plan_ix)
             )

    # optimize and return dual info
    optimize!(model)
    
    return model, dual.(plan_per_fire), dual.(route_per_crew), dual.(cover_plans)

end

run_master_problem_step (generic function with 1 method)

In [29]:
function add_route_plans(arc_data, sigma, rho, sp_models, crew_route_count_dict, A, route_costs)
    
    # for each crew
    for crew in 1:NUM_CREWS
        
        # solve subproblem
        local_cost, assignments, pure_cost = route_subproblem(arc_data, crew, sp_models[crew], rho)
        
        # if negatve reduced cost
        if local_cost < sigma[crew] - THRESH
            
            # add plan and cost
            num_routes = crew_route_count_dict[crew]
            A, route_costs = update_crew_route_data(A, route_costs, crew, num_routes, assignments, pure_cost)
            crew_route_count_dict[crew] += 1
            
        end
    end
    
    return A, route_costs, crew_route_count_dict
end

add_route_plans (generic function with 1 method)

In [30]:
function add_suppression_plans(fire_data, pi, rho, sp_models, fire_plan_count_dict, B, fire_costs, second_objective_eps)
    
    for fire in 1:NUM_FIRES
        
        plan, pure_cost, local_cost = suppression_plan_subproblem(fire, sp_models[fire], rho, second_objective_eps)
        
        # if negatve reduced cost
        if local_cost < pi[fire] - THRESH
            
            # add plan and cost
            num_plans = fire_plan_count_dict[fire]
            update_suppression_plan_data(fire_data, B, fire_costs, fire, num_plans, plan)
            fire_plan_count_dict[fire] += 1
        end
    end
    
    return B, fire_costs, fire_plan_count_dict
end

add_suppression_plans (generic function with 1 method)

In [31]:
function solve_integer_MP(crew_route_count_dict, fire_plan_count_dict, route_costs, fire_costs, A, B)
    
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    # make variable index sets
    fire_plan_ix = [(i, g) for g=1:NUM_FIRES for i=1:fire_plan_count_dict[g]]
    route_plan_ix = [(i, c) for c=1:NUM_CREWS for i=1:crew_route_count_dict[c]]
    
    
    # formulate "disappointingly small" formulation
    @variable(model, suppression_plans[fire_plan_ix] >= 0, Int)
    @variable(model, crew_routes[route_plan_ix] >= 0, Int)

    @constraint(model, plan_per_fire[g=1:NUM_FIRES], 
                    sum(suppression_plans[ix] for ix in fire_plan_ix if ix[2] == g) >= 1)
    @constraint(model, route_per_crew[c=1:NUM_CREWS], 
                    sum(crew_routes[ix] for ix in route_plan_ix if ix[2] == c) == 1)
    @constraint(model, cover_plans[g=1:NUM_FIRES, t=1:NUM_TIME_PERIODS],
                    sum(crew_routes[ix] * A[ix[1], ix[2], g, t] for ix in route_plan_ix) >=
                    sum(suppression_plans[ix] * B[ix[1], g, t] for ix in fire_plan_ix if ix[2] == g)
               )
    @objective(model, Min, 
                  sum(crew_routes[ix] * route_costs[ix[1], ix[2]] for ix in route_plan_ix) + 
                  sum(suppression_plans[ix] * fire_costs[ix[1], ix[2]] for ix in fire_plan_ix)
             )

    # optimize and return dual info
    optimize!(model)
    
    supp = zeros((NUM_FIRES, NUM_TIME_PERIODS))
    for g=1:NUM_FIRES
        for t=1:NUM_TIME_PERIODS
            supp[g, t] = sum(value.(suppression_plans[ix] * B[ix[1], g, t] for ix in fire_plan_ix if ix[2] == g))
        end
    end
    
    crew_routes_chosen = convert.(Int, zeros(NUM_CREWS))
    for tuple in [key for key in route_plan_ix if value(crew_routes[key]) > 0.99]
        crew_routes_chosen[tuple[2]] = tuple[1]
    end
    
    fire_plans_chosen = convert.(Int, zeros(NUM_FIRES)) 
    for tuple in [key for key in fire_plan_ix if value(suppression_plans[key]) > 0.99]
        fire_plans_chosen[tuple[2]] = tuple[1]
    end
    return model, supp, fire_plans_chosen, crew_routes_chosen              

end

solve_integer_MP (generic function with 1 method)

In [32]:
function run_CG(arc_data, fire_data, constraint_data, warm_start_routes_per_crew, warm_start_supp_plans_per_fire, verbose)

    # initialize CG model and stats to track #
    formulate_time = @elapsed fire_subproblem_models = 
                                   [init_suppression_plan_subproblem(fire_data, fire) for fire=1:NUM_FIRES]
    formulate_time += @elapsed route_subproblem_models = [init_route_subproblem(arc_data, constraint_data, crew) 
                                                          for crew=1:NUM_CREWS]
    
    # solve LP relaxation of original formulaton #
    
    relaxation_time = @elapsed m, p, l, ff, fr, rf, rr = full_formulation(false, arc_data, fire_data, constraint_data)
    relaxation_time += @elapsed optimize!(m)
    verbose && println("Seconds for solving LP relaxation: $(relaxation_time)")

    # warm start

    fire_warm_start_time = @elapsed B, fire_plan_costs, num_plans_per_fire = 
        create_warm_start_suppression_plans(fire_data, l, warm_start_supp_plans_per_fire)
    verbose && println("Seconds for generating $(sum(num_plans_per_fire)) warm-start fire plans: $(fire_warm_start_time)")
    

    crew_warm_start_time = @elapsed A, crew_plan_costs, num_plans_per_crew = 
        create_warm_start_crew_routes(arc_data, constraint_data, B[1:warm_start_supp_plans_per_fire, :, :], warm_start_routes_per_crew)
    verbose && println("Seconds for generating $(sum(num_plans_per_crew)) warm-start crew routes: $(crew_warm_start_time)")
    
#    crew_warm_start_time = @elapsed A, crew_plan_costs, num_plans_per_crew = 
#        fast_create_warm_start_crew_routes(arc_data, constraint_data, B[1, :, :], route_subproblem_models, 
#                                           value.(ff), value.(fr), value.(rf), value.(rr))
#    verbose && println("Seconds for generating $(sum(num_plans_per_crew)) warm-start crew routes: $(crew_warm_start_time)")
    
    initial_num_fire_plans = copy(num_plans_per_fire)
    initial_num_crew_plans = copy(num_plans_per_crew)



    
    last_total_crew_plans = -1
    last_total_fire_plans = -1

    total_crew_plans = sum(num_plans_per_crew)
    total_fire_plans = sum(num_plans_per_fire)

    n_iter = 0
    
    fire_subproblem_times = []
    crew_subproblem_times = []
    mp_times = []
    
    
    ws_obj = 0

    mp_model = []
    
    crew_plan_histories = []
    fire_plan_histories = []
    
    progress = []
    
    # run CG algorithm until proof of optimality #
    
    while (total_crew_plans != last_total_crew_plans) | (total_fire_plans != last_total_fire_plans)

        push!(crew_plan_histories, copy(num_plans_per_crew))
        push!(fire_plan_histories, copy(num_plans_per_fire))
        n_iter += 1

        last_total_crew_plans = total_crew_plans
        last_total_fire_plans = total_fire_plans

        time = @elapsed mp_model, p, s, r = run_master_problem_step(num_plans_per_crew, 
                                                                    num_plans_per_fire, 
                                                                    crew_plan_costs, 
                                                                    fire_plan_costs,
                                                                    A, B)
        push!(mp_times, time)
        
        push!(progress, objective_value(mp_model))
        if n_iter == 1
            ws_obj = objective_value(mp_model)
        end
        
        time = @elapsed A, crew_plan_costs, num_plans_per_crew  = 
                add_route_plans(arc_data, s, r, route_subproblem_models, num_plans_per_crew, A, crew_plan_costs)
        push!(fire_subproblem_times, time)

        
        time = @elapsed B, fire_plan_costs, num_plans_per_fire  = 
                add_suppression_plans(fire_data, p, r, fire_subproblem_models, num_plans_per_fire, B, fire_plan_costs, 0.01)
        push!(crew_subproblem_times, time)
        
        total_crew_plans = sum(num_plans_per_crew)
        total_fire_plans = sum(num_plans_per_fire)

    end
    
    final_num_fire_plans = copy(num_plans_per_fire)
    final_num_crew_plans = copy(num_plans_per_crew)
    
    verbose && println("Seconds spent on master problem: $(sum(mp_times))")
    verbose && println("Seconds spent on fire subproblems: $(sum(fire_subproblem_times))")
    verbose && println("Seconds spent on crew subproblems: $(sum(crew_subproblem_times))")
    
    int_problem = @elapsed model, supp, fire_plans_chosen, crew_routes_chosen = 
                            solve_integer_MP(num_plans_per_crew, 
                            num_plans_per_fire, 
                            crew_plan_costs, 
                            fire_plan_costs,
                            A, B);
    verbose && println("Seconds spent on restoring integrality: $(int_problem)")
    
    Dict("obj_LR" => objective_value(m), "obj_MP" => objective_value(mp_model), "obj_PB" => objective_value(model),
         "obj_MP_warm_start_only" => ws_obj,
         "time_LR" => relaxation_time, "time_formulate" => formulate_time, "time_fire_warm_start" => fire_warm_start_time, 
         "time_crew_warm_start" => crew_warm_start_time, "iterations" => n_iter, "mp_iter_times" => mp_times,
         "fire_sp_times" => fire_subproblem_times, "crew_sp_times" => crew_subproblem_times,
         "restore_int_time" => int_problem,
         "fire_plans_chosen" => fire_plans_chosen, "crew_routes_chosen" => crew_routes_chosen,
         "initial_fire_plans" => initial_num_fire_plans, "initial_crew_routes" => initial_num_crew_plans,
         "final_fire_plans" => final_num_fire_plans, "final_crew_routes" => final_num_crew_plans,
         "supp_LR" => value.(l) / LINE_PER_CREW, "supp_PB" => supp, "num_plans_per_crew" => num_plans_per_crew,
         "num_plans_per_fire" => num_plans_per_fire, "crew_plan_costs" => crew_plan_costs, 
         "fire_plan_costs" => fire_plan_costs, "crew_plan_histories" => crew_plan_histories, "fire_plan_histories" => 
          fire_plan_histories, "progress" => progress)
end

run_CG (generic function with 1 method)

In [33]:
THRESH = 0.001

0.001

In [34]:
ENV["COLUMNS"] = 200

200

In [35]:
function analysis(arc_data, fire_data, constraint_data, ws_params)
    
    # solve completely with Gurobi
    formulate_time = @elapsed m, _, l, _, _, _, _ = full_formulation(true, arc_data, fire_data, constraint_data)
    opt_time = @elapsed optimize!(m)
    obj_IP = objective_value(m)
    obj_PB = false
    IP_sol = round.(value.(l) / LINE_PER_CREW)
    
    times = []
    crew_ws_nums = []
    fire_ws_nums = []
    iters = []
    fire_ws_times = []
    crew_ws_times = []
    mp_times = []
    lr_times = []
    formulate_times = []
    fire_sp_times = []
    crew_sp_times = []
    
    PB_IP_opt_gaps = []
    MP_LR_opt_gaps = []
    MPWS_opt_gaps = []
    fire_plans_generated = []
    crew_routes_generated = []
    ws_fire_plans_used = []
    ws_crew_routes_used = []
    sp_fire_plans_used = []
    sp_crew_routes_used = []
    
    
    # for each set of warm start parameters
    for ws_param in ws_params
        
        time = @elapsed result = run_CG(arc_data, fire_data, constraint_data, ws_param[1], ws_param[2], false)
        
        obj_PB = result["obj_PB"]
        push!(times, time)
        push!(crew_ws_nums, ws_param[1])
        push!(fire_ws_nums, ws_param[2])
        push!(iters, result["iterations"])
        push!(lr_times, result["time_LR"])
        push!(formulate_times, result["time_formulate"])
        push!(fire_ws_times, sum(result["time_fire_warm_start"]))
        push!(crew_ws_times, sum(result["time_crew_warm_start"]))
        push!(mp_times, sum(result["mp_iter_times"]))
        push!(fire_sp_times, sum(result["fire_sp_times"]))
        push!(crew_sp_times, sum(result["crew_sp_times"]))
        
        push!(PB_IP_opt_gaps, (result["obj_PB"] - obj_IP) / obj_IP)
        push!(MP_LR_opt_gaps, (result["obj_MP"] - result["obj_LR"]) / result["obj_LR"])
        push!(MPWS_opt_gaps, (result["obj_MP_warm_start_only"] - result["obj_MP"]) / result["obj_MP"])
        push!(fire_plans_generated, sum(result["final_fire_plans"]) - sum(result["initial_fire_plans"]))
        push!(crew_routes_generated, sum(result["final_crew_routes"]) - sum(result["initial_crew_routes"]))
        push!(sp_fire_plans_used, sum(result["fire_plans_chosen"] .> result["initial_fire_plans"]))
        push!(ws_fire_plans_used, sum(result["fire_plans_chosen"] .<= result["initial_fire_plans"]))
        push!(sp_crew_routes_used, sum(result["crew_routes_chosen"] .> result["initial_crew_routes"]))
        push!(ws_crew_routes_used, sum(result["crew_routes_chosen"] .<= result["initial_crew_routes"]))
        
    end
    
    times = DataFrame(ws_crew = crew_ws_nums, ws_fire = fire_ws_nums,
                      iterations = iters, time = times, lr_time = lr_times, formulate_sp_time = formulate_times,
                      fire_ws_time = fire_ws_times, 
                      crew_ws_time = crew_ws_times, mp_time = mp_times, fire_sp_time = fire_sp_times, 
                      crew_sp_time = crew_sp_times)
    
    plans = DataFrame(ws_crew = crew_ws_nums, ws_fire = fire_ws_nums,
                      iterations = iters, PB_IP_opt_gap = PB_IP_opt_gaps, MP_LR_opt_gap = MP_LR_opt_gaps,
                      MPWS_MPLR_opt_gap = MPWS_opt_gaps,
                      total_fire_plans_sp = fire_plans_generated, total_crew_routes_sp = crew_routes_generated,
                      sp_fire_plans_used = sp_fire_plans_used, ws_fire_plans_used = ws_fire_plans_used,
                      sp_crew_routes_used = sp_crew_routes_used, ws_crew_routes_used = ws_crew_routes_used)
    
    return formulate_time, opt_time, obj_IP, obj_PB, IP_sol, times, plans
    
end

analysis (generic function with 1 method)

In [163]:
function read_inputs(in_path, init_path)
    
    # get inital fire perimeters and no-suppression progression parameters
    M = readdlm(init_path * "/sample_growth_patterns.csv", ',')[1:NUM_FIRES, :]
    start_perims = M[:, 1]
    progressions = M[:, 2:NUM_TIME_PERIODS+1]     

    # get distance from fire f to fire g 
    fire_dists =  readdlm(in_path * "/fire_distances.csv", ',')

    # get distance from base c to fire g (NUM_CREWS-by-NUM_FIRES)
    base_fire_dists =  readdlm(in_path * "/base_fire_distances.csv", ',')

    # initialize travel times (number of periods) from fire f to fire g
    tau = convert.(Int, round.(fire_dists ./ 500) .+ 1)

    # initialize number of periods to travel from base c to fire g (NUM_CREWS-by-NUM_FIRES)
    tau_base_to_fire = convert.(Int, round.(base_fire_dists ./ 500) .+ 1)

    # read intial crew statuses (location, period by which they must rest)
    # (-1 in current_fire means crew is currently at base)
    # (rested_periods is the amount of time crew has been at base, relevant for completing rest)
    crew_starts = CSV.read(init_path * "/sample_crew_starts.csv", DataFrame)[1:NUM_CREWS, :]
    rest_by = crew_starts[!, "rest_by"]
    current_fire = crew_starts[!, "current_fire"]
    rested_periods = crew_starts[!, "rested_periods"]
    
    start_perims, progressions, fire_dists, base_fire_dists, tau, tau_base_to_fire, rest_by, current_fire, rested_periods
end

read_inputs (generic function with 1 method)

In [37]:
function fast_warm_start(arc_data, ff_val, fr_val, rf_val, rr_val)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)


    @variable(m, ff[arc_data.ff_ix] >= 0)
    @variable(m, fr[arc_data.fr_ix] >= 0)
    @variable(m, rf[arc_data.rf_ix] >= 0)
    @variable(m, rr[arc_data.rr_ix] >= 0)



    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=0:1],

                # outflow
                sum(ff[key] for key in arc_data.ff_ix_arr[c]
                        if (key[2] == g) & (key[4] == t) & (key[6] == rest)
                    ) +    

                sum(fr[key] for key in arc_data.fr_ix_arr[c]
                            if (key[2] == g) & (key[3] == t) & (key[5] == rest)
                    ) 

        ==
                # inflow
                sum(ff[key] for key in arc_data.ff_ix_arr[c]
                            if (key[3] == g) & (key[5] == t) & (key[6] == rest)
                    ) +

                sum(rf[key] for key in arc_data.rf_ix_arr[c]
                            if (key[2] == g) & (key[4] == t) & (key[5] == rest)
                    ) 

                )   

    @constraint(m, rest_flow[c=1:C, t=1:T, rest=0:1], 

                # outflow
                sum(rf[key] for key in arc_data.rf_ix_arr[c]
                            if (key[3] == t) & (key[5] == rest)
                    ) +

                sum(rr[key] for key in arc_data.rr_ix_arr[c]
                            if (key[2] == t) & (key[4] == rest)
                    )

                ==       

                # inflow
                sum(fr[key] for key in arc_data.fr_ix_arr[c]
                            if (key[4] == t) &  (key[5] == rest)
                    ) +
                sum(rr[key] for key in arc_data.rr_ix_arr[c]
                            if (key[3] == t) & (key[5] == rest)
                    )
               )

    @constraint(m, start[c=1:C], 

        sum(ff[key] for key in arc_data.from_start_ff if key[1] == c) + 
        sum(rf[key] for key in arc_data.from_start_rf if key[1] == c) + 
        sum(fr[key] for key in arc_data.from_start_fr if key[1] == c) + 
        sum(rr[key] for key in arc_data.from_start_rr if key[1] == c) == 1
               )

    
    @objective(m, Max,
    sum(ff[key] * ff_val[key] for key in arc_data.ff_ix) +
    sum(fr[key] * fr_val[key] for key in arc_data.fr_ix) + 
    sum(rf[key] * rf_val[key] for key in arc_data.rf_ix) + 
    sum(rr[key] * rr_val[key] for key in arc_data.rr_ix)
        )
    
    optimize!(m)
    
    fires_fought = vcat([(ix[1], ix[3], ix[5]) for ix in arc_data.ff_ix if (value(ff[ix]) > 0.99)],
                [(ix[1], ix[2], ix[4]) for ix in arc_data.rf_ix if (value(rf[ix]) > 0.99)])
    
    route_costs = 
    [sum(value(ff[key]) * (ALPHA + fire_dists[key[2], key[3]]) for key in arc_data.ff_ix_arr[crew]) +
    sum(value(fr[key]) * (base_fire_dists[key[1], key[2]]) for key in arc_data.fr_ix_arr[crew]) + 
    sum(value(rf[key]) * (ALPHA + base_fire_dists[key[1], key[2]]) for key in arc_data.rf_ix_arr[crew])
        for crew = 1:C]
    
    fires_fought, route_costs

end

fast_warm_start (generic function with 1 method)

In [176]:
# set path to all input data
in_path = "data/processed/big_fires"

# set path to intialization data
init_path = "data/raw/big_fire"

baseline_costs = [[], [], [], [], [], []]
baseline_supps = [[], [], [], [], [], []];

In [182]:
NUM_CREWS = 100
NUM_FIRES = 6
LINE_PER_CREW = 4

4

In [183]:
start_perims, progressions, fire_dists, base_fire_dists, tau, tau_base_to_fire, rest_by, current_fire, rested_periods =
     read_inputs(in_path, init_path)

arcs = define_arc_sets(tau, tau_base_to_fire, current_fire, rest_by, rested_periods)
fire_progs = FireProgressionData(start_perims, progressions)
constraints = define_network_constraint_data(arcs);

In [188]:
m, p, l, ff, fr, rf, rr = full_formulation(true, arcs, fire_progs, constraints);

In [189]:
time = @elapsed optimize!(m)
objective_value(m)

4.0196488809042843e6

In [190]:
time

30.4136523

In [45]:
time

17.9045187

In [46]:
@elapsed result = run_CG(arcs, fire_progs, constraints, 1, 20, false)

31.1602772

In [47]:
num_arcs = size(arcs.ff_ix)[1] + size(arcs.fr_ix)[1] + size(arcs.rf_ix)[1] + size(arcs.rr_ix)[1]

21364

In [215]:
timings = []

limit = 1800

params = [(100, 15, 6)]

for param in params
    crew = param[1]
    fire = param[2]
    NUM_CREWS = crew
    NUM_FIRES = fire
    LINE_PER_CREW = param[3]
    start_perims, progressions, fire_dists, base_fire_dists, tau, tau_base_to_fire, rest_by, current_fire, rested_periods =
         read_inputs(in_path, init_path)

    arcs = define_arc_sets(tau, tau_base_to_fire, current_fire, rest_by, rested_periods)
    fire_progs = FireProgressionData(start_perims, progressions)
    constraints = define_network_constraint_data(arcs)

    d = test_cg_time(arcs, fire_progs, constraints, 2, 20, limit)
    true_time = (d["time_LR"] + d["time_fire_warm_start"] + d["time_crew_warm_start"] + d["time_formulate"] + d["cg_time"] 
    + d["restore_int_time"])

    to_optimality, feasible_solution, lb, ub, gap, formulate_time, solve_time = test_gurobi_time(arcs, fire_progs, constraints, 
                                                                                                 maximum([true_time, limit]))

    push!(timings, copy([(to_optimality, feasible_solution, lb, ub, gap, formulate_time, solve_time),
             (d["time_LR"], d["obj_LR"], d["time_fire_warm_start"], d["time_crew_warm_start"], d["time_formulate"],
              d["cg_time"], d["restore_int_time"], d["obj_PB"]), (crew, fire)]))
end

In [216]:
timings

1-element Vector{Any}:
 Tuple{Real, Real, Vararg{Float64}}[(true, true, 1.9388969335930556e7, 1.9390790077221803e7, 9.389722048438437e-5, 2.644157, 365.8492238), (16.9094366, 1.9384726773417536e7, 1.5983169, 763.9762755, 1.5735071, 1006.6861151000003, 1.5298293, 1.945339024033259e7), (100, 15)]

In [217]:
gurobi_lb = []
gurobi_ub = []
gurobi_solved = []
gurobi_time = []
cg_lb = []
cg_ub = []
cg_time = []
crews = []
fires = []

for time in timings
    push!(crews, time[3][1])
    push!(fires, time[3][2])
    
    push!(gurobi_lb, time[1][3])
    push!(gurobi_ub, time[1][4])
    push!(gurobi_solved, time[1][1])
    push!(gurobi_time, time[1][6] + time[1][7])
    
    push!(cg_lb, time[2][2])
    push!(cg_ub, time[2][8])
    push!(cg_time, sum(time[2][[1, 3, 4, 6, 7]]))
end

In [218]:
DataFrame(Dict("crews" => crews, "fires" => fires, "gurobi_time" => gurobi_time,
               "gurobi_solved" => gurobi_solved, "gurobi_ub" => gurobi_ub, "gurobi_lb" => gurobi_lb,
               "price_and_branch_time" => cg_time, "price_and_branch_ub" => cg_ub, "price_and_branch_lb" => cg_lb))

,crews,fires,gurobi_lb,gurobi_solved,gurobi_time,gurobi_ub,price_and_branch_lb,price_and_branch_time,price_and_branch_ub
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,100,15,1.9389e7,1,368.493,1.93908e7,1.93847e7,1790.7,1.94534e7


In [133]:
df = DataFrame(Dict("crews" => crews, "fires" => fires, "gurobi_time" => gurobi_time,
               "gurobi_solved" => gurobi_solved, "gurobi_ub" => gurobi_ub, "gurobi_lb" => gurobi_lb,
               "price_and_branch_time" => cg_time, "price_and_branch_ub" => cg_ub, "price_and_branch_lb" => cg_lb));

In [134]:
df = copy(df)

,crews,fires,gurobi_lb,gurobi_solved,gurobi_time,gurobi_ub,price_and_branch_lb,price_and_branch_time,price_and_branch_ub
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,20,6,2.86861e6,1,11.606,2.86878e6,2.80458e6,37.7273,2.93314e6
2,24,7,2.52406e6,1,36.4784,2.52425e6,2.45629e6,23.6031,2.61101e6
3,28,8,3.85427e6,1,186.711,3.85464e6,3.74822e6,37.5499,4.11547e6
4,30,10,5.61989e6,0,1800.78,5.62345e6,5.53247e6,97.8495,5.67499e6
5,20,10,9.8647e6,1,54.2828,9.86565e6,9.72177e6,65.9624,1.01527e7
6,24,13,1.2081e7,1,231.937,1.20813e7,1.17972e7,136.272,1.21891e7
7,28,18,2.24302e7,0,1801.69,2.24361e7,2.21592e7,289.444,2.24897e7
8,30,20,2.42107e7,0,1802.64,2.42147e7,2.39363e7,489.462,2.42974e7


In [143]:
fire_dists[:, 1]

105-element Vector{Float64}:
   0.0
 125.23371166147112
 351.30016167042
  58.105269117111256
 109.5362310668155
  46.57186521725672
  14.923837537074276
 140.3479950832503
  55.280941514676485
 521.2741618441719
  77.91103351722482
   7.7263295271719254
 137.26427526047797
   ⋮
 315.9102624068898
  52.009257771456824
 504.97397500952775
 801.4167088423508
 215.16241020794857
 126.07486679640888
 939.2808872436176
  95.16446831722033
 393.4802324749388
  15.606842935960508
  76.97433281643126
 347.92217320779946

In [139]:
100 * (df[!, "gurobi_ub"] ./ df[!, "price_and_branch_ub"] .- 1)

8-element Vector{Float64}:
 -2.1942932428638096
 -3.322849411156159
 -6.337632432128859
 -0.9081942570679957
 -2.826850732335129
 -0.8848841917869499
 -0.2383963705964942
 -0.34062669281279545

In [98]:
gurobi_lb

1-element Vector{Any}:
 -1.0e100

In [99]:
gurobi_time

1-element Vector{Any}:
 35.6689751

In [37]:
f = [myopic_unsuppressed_size, unsuppressed_size, myopic_unsuppressed_damage, unsuppressed_damage]
for i in 1:length(f) 
    supp, cost = baseline(f[i])
    push!(baseline_supps[i], supp)
    push!(baseline_costs[i], cost)
end

LoadError: UndefVarError: myopic_unsuppressed_size not defined

In [116]:
params = [(1, 1), (1, 5), (1, 10), (3, 3), (3, 10), (5, 10), (10, 20), (20, 20)]
params = [(1, 1), (1, 20), (20, 20)]
ip_formulate_time, ip_solve_time, obj_IP, obj_PB, ip_sol, times, plans = analysis(arcs, fire_progs, constraints, params)

ip_formulate_time, ip_solve_time

(0.0722535, 11.3181634)

In [117]:
push!(baseline_supps[5], convert.(Int, ip_sol))
push!(baseline_costs[5], obj_IP)
push!(baseline_costs[6], obj_PB);

In [118]:
ip_sol

6×14 Matrix{Float64}:
 0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0  0.0  12.0  11.0  0.0  0.0  0.0
 0.0   0.0   0.0   4.0   4.0  5.0   5.0   5.0  6.0   1.0   0.0  0.0  0.0  0.0
 9.0   2.0   0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0
 7.0  16.0  17.0   3.0   0.0  0.0   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0
 0.0   0.0   0.0  10.0  10.0  9.0  10.0  10.0  9.0   1.0   0.0  0.0  0.0  0.0
 2.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0   2.0  0.0  0.0  0.0

In [119]:
times

,ws_crew,ws_fire,iterations,time,lr_time,formulate_sp_time,fire_ws_time,crew_ws_time,mp_time,fire_sp_time,crew_sp_time
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,1,75,20.0862,0.322616,0.0448695,0.72508,0.248657,1.09301,8.25436,9.3119
2,1,20,69,17.98,0.310035,0.0878993,0.663264,0.253924,1.05829,8.35816,7.00928
3,20,20,71,27.8657,0.351853,0.0441006,0.657116,7.74996,1.37564,7.69404,9.88622


In [120]:
plans

,ws_crew,ws_fire,iterations,PB_IP_opt_gap,MP_LR_opt_gap,MPWS_MPLR_opt_gap,total_fire_plans_sp,total_crew_routes_sp,sp_fire_plans_used,ws_fire_plans_used,sp_crew_routes_used,ws_crew_routes_used
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,1,75,0.0547889,0.022522,35.6183,320,1164,1,5,20,0
2,1,20,69,0.0222131,0.0225218,35.6183,297,1228,0,6,20,0
3,20,20,71,0.0221756,0.022522,0.201157,255,1167,0,6,10,10


In [68]:
NUM_CREWS = 20    
NUM_FIRES = 6

6

In [69]:
start_perims, progressions, fire_dists, base_fire_dists, tau, tau_base_to_fire, rest_by, current_fire, rested_periods =
     read_inputs(in_path, init_path)

arcs = define_arc_sets(tau, tau_base_to_fire, current_fire, rest_by, rested_periods)
fire_progs = FireProgressionData(start_perims, progressions);

In [106]:
f = [myopic_unsuppressed_size, unsuppressed_size, myopic_unsuppressed_damage, unsuppressed_damage]
for i in 1:length(f) 
    supp, cost = baseline(f[i])
    push!(baseline_supps[i], supp)
    push!(baseline_costs[i], cost)
end

In [52]:
# params = [(1, 1), (1, 5), (1, 10), (1, 20), (3, 20), (10, 20)]
params = [(1, 20)]
ip_formulate_time, ip_solve_time, obj_IP, obj_PB, ip_sol, times, plans = analysis(arcs, fire_progs, params)

ip_formulate_time, ip_solve_time

(8.3789707, 15.6629682)

In [53]:
push!(baseline_supps[5], convert.(Int, ip_sol))
push!(baseline_costs[5], obj_IP)
push!(baseline_costs[6], obj_PB);

In [54]:
obj_PB

2.9325036370976088e6

In [55]:
ip_sol

6×14 Matrix{Float64}:
 0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0  0.0  12.0  11.0  0.0  0.0  0.0
 0.0   0.0   0.0   4.0   4.0  5.0   5.0   5.0  6.0   1.0   0.0  0.0  0.0  0.0
 9.0   2.0   0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0
 7.0  16.0  17.0   3.0   0.0  0.0   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0
 0.0   0.0   0.0  10.0  10.0  9.0  10.0  10.0  9.0   1.0   0.0  0.0  0.0  0.0
 2.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0   2.0  0.0  0.0  0.0

In [56]:
times

,ws_crew,ws_fire,iterations,time,lr_time,formulate_sp_time,fire_ws_time,crew_ws_time,mp_time,fire_sp_time,crew_sp_time
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,20,69,45.6953,9.13566,8.41734,0.741509,9.95021,1.21654,8.69661,7.24978


In [57]:
plans

,ws_crew,ws_fire,iterations,PB_IP_opt_gap,MP_LR_opt_gap,MPWS_MPLR_opt_gap,total_fire_plans_sp,total_crew_routes_sp,sp_fire_plans_used,ws_fire_plans_used,sp_crew_routes_used,ws_crew_routes_used
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,20,69,0.0222131,0.0225218,35.6183,297,1228,0,6,20,0


In [58]:
NUM_CREWS = 24     
NUM_FIRES = 7

7

In [59]:
start_perims, progressions, fire_dists, base_fire_dists, tau, tau_base_to_fire, rest_by, current_fire, rested_periods =
     read_inputs(in_path, init_path)

arcs = define_arc_sets(tau, tau_base_to_fire, current_fire, rest_by, rested_periods)
fire_progs = FireProgressionData(start_perims, progressions);

In [60]:
f = [myopic_unsuppressed_size, unsuppressed_size, myopic_unsuppressed_damage, unsuppressed_damage]
for i in 1:length(f) 
    supp, cost = baseline(f[i])
    push!(baseline_supps[i], supp)
    push!(baseline_costs[i], cost)
end

In [61]:
# params = [(1, 1), (1, 5), (1, 10), (1, 20), (3, 20), (10, 20)]
params = [(1, 20)]
ip_formulate_time, ip_solve_time, obj_IP, obj_PB, ip_sol, times, plans = analysis(arcs, fire_progs, params)

ip_formulate_time, ip_solve_time

(17.2755852, 40.196114)

In [62]:
push!(baseline_supps[5], convert.(Int, ip_sol))
push!(baseline_costs[5], obj_IP)
push!(baseline_costs[6], obj_PB);

In [63]:
ip_sol

7×14 Matrix{Float64}:
 0.0   0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  6.0  15.0  0.0  0.0  0.0
 0.0   0.0   0.0  5.0  4.0  5.0   5.0  5.0  4.0  2.0   0.0  0.0  0.0  0.0
 9.0   2.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0
 4.0  20.0  21.0  3.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0
 0.0   0.0   0.0  9.0  9.0  9.0  10.0  8.0  7.0  6.0   1.0  0.0  0.0  0.0
 9.0   0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0
 0.0   0.0   0.0  4.0  5.0  4.0   4.0  5.0  6.0  2.0   0.0  0.0  0.0  0.0

In [64]:
times

,ws_crew,ws_fire,iterations,time,lr_time,formulate_sp_time,fire_ws_time,crew_ws_time,mp_time,fire_sp_time,crew_sp_time
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,20,53,66.595,17.3541,14.8763,0.861783,17.1879,1.06395,11.3056,3.84309


In [65]:
plans

,ws_crew,ws_fire,iterations,PB_IP_opt_gap,MP_LR_opt_gap,MPWS_MPLR_opt_gap,total_fire_plans_sp,total_crew_routes_sp,sp_fire_plans_used,ws_fire_plans_used,sp_crew_routes_used,ws_crew_routes_used
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,20,53,0.0196582,0.0248746,41.8171,277,1150,1,6,24,0


In [663]:
NUM_CREWS = 30     
NUM_FIRES = 9

9

In [664]:
start_perims, progressions, fire_dists, base_fire_dists, tau, tau_base_to_fire, rest_by, current_fire, rested_periods =
     read_inputs(in_path, init_path)

arcs = define_arc_sets(tau, tau_base_to_fire, current_fire, rest_by, rested_periods)
fire_progs = FireProgressionData(start_perims, progressions);

In [665]:
f = [myopic_unsuppressed_size, unsuppressed_size, myopic_unsuppressed_damage, unsuppressed_damage]
for i in 1:length(f) 
    supp, cost = baseline(f[i])
    push!(baseline_supps[i], supp)
    push!(baseline_costs[i], cost)
end

In [ ]:
# params = [(1, 1), (1, 5), (1, 10), (1, 20), (3, 20), (10, 20)]
params = [(1, 20)]
ip_formulate_time, ip_solve_time, obj_IP, obj_PB, ip_sol, times, plans = analysis(arcs, fire_progs, params)

ip_formulate_time, ip_solve_time

In [ ]:
push!(baseline_supps[5], convert.(Int, ip_sol))
push!(baseline_costs[5], obj_IP)
push!(baseline_costs[6], obj_PB);

In [ ]:
ip_sol

In [ ]:
times

In [ ]:
plans

In [66]:
df = DataFrame(optimal = baseline_costs[5], price_branch = baseline_costs[6], myopic_biggest = baseline_costs[1], unsuppressed_biggest = baseline_costs[2],
               myopic_most_damage = baseline_costs[3], unsuppressed_most_damage = baseline_costs[4]
)

percent_gap = copy(df)
for col in names(df)
    percent_gap[!, col] = 100 * (df[!, col] .- df[!, "optimal"]) ./ df[!, "optimal"]
end

percent_gap

,optimal,price_branch,myopic_biggest,unsuppressed_biggest,myopic_most_damage,unsuppressed_most_damage
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0956726,6875.62,6.89521,6318.42,7.77077
2,0.0,2.22131,2751.33,88.8431,2596.32,91.5088
3,0.0,1.96582,3176.49,130.244,2983.03,128.988


In [49]:
function test_gurobi_time(arc_data, fire_data, constraint_data, time_limit_secs)
    
    formulate_time = @elapsed m, p, l, ff, fr, rf, rr = full_formulation(true, arc_data, fire_data, constraint_data)
    set_optimizer_attribute(m, "TimeLimit", max(1, time_limit_secs - formulate_time))
    
    solve_time = @elapsed optimize!(m)
    to_optimality = (termination_status(m) == MOI.OPTIMAL)

    feasible_solution = has_values(m)
    lb = objective_bound(m)
    
    ub = false
    gap = false
    if feasible_solution
        ub = objective_value(m)
        gap = relative_gap(m)
    end
    
    return to_optimality, feasible_solution, lb, ub, gap, formulate_time, solve_time
end 

test_gurobi_time (generic function with 1 method)

In [112]:
function test_cg_time(arc_data, fire_data, constraint_data, warm_start_routes_per_crew, warm_start_supp_plans_per_fire, time_limit_secs)

    # initialize CG model and stats to track #
    formulate_time = @elapsed fire_subproblem_models = 
                                   [init_suppression_plan_subproblem(fire_data, fire) for fire=1:NUM_FIRES]
    formulate_time += @elapsed route_subproblem_models = [init_route_subproblem(arc_data, constraint_data, crew) for crew=1:NUM_CREWS]
    
    # solve LP relaxation of original formulaton #
    
    relaxation_time = @elapsed m, p, l, ff, fr, rf, rr = full_formulation(false, arc_data, fire_data, constraint_data)
    relaxation_time += @elapsed optimize!(m)

    # warm start

    fire_warm_start_time = @elapsed B, fire_plan_costs, num_plans_per_fire = 
        create_warm_start_suppression_plans(fire_data, l, warm_start_supp_plans_per_fire)
    
    crew_warm_start_time = @elapsed A, crew_plan_costs, num_plans_per_crew = 
        create_warm_start_crew_routes(arc_data, constraint_data, B[1:warm_start_supp_plans_per_fire, :, :], warm_start_routes_per_crew)
      
    initial_num_fire_plans = copy(num_plans_per_fire)
    initial_num_crew_plans = copy(num_plans_per_crew)

    last_total_crew_plans = -1
    last_total_fire_plans = -1

    total_crew_plans = sum(num_plans_per_crew)
    total_fire_plans = sum(num_plans_per_fire)

    n_iter = 0
    
    fire_subproblem_times = []
    crew_subproblem_times = []
    
    crew_plan_histories = []
    fire_plan_histories = []
    
    mp_model = []
    
    
    time = formulate_time + relaxation_time + fire_warm_start_time + crew_warm_start_time
    
    # run CG algorithm until proof of optimality #
    
    while (time < time_limit_secs) & (maximum(num_plans_per_crew) < 1000) & (maximum(num_plans_per_fire) < 1000) &
          ((total_crew_plans != last_total_crew_plans) | (total_fire_plans != last_total_fire_plans))


        n_iter += 1

        last_total_crew_plans = total_crew_plans
        last_total_fire_plans = total_fire_plans

        time += @elapsed mp_model, p, s, r = run_master_problem_step(num_plans_per_crew, 
                                                                    num_plans_per_fire, 
                                                                    crew_plan_costs, 
                                                                    fire_plan_costs,
                                                                    A, B)

        time += @elapsed A, crew_plan_costs, num_plans_per_crew  = 
                add_route_plans(arc_data, s, r, route_subproblem_models, num_plans_per_crew, A, crew_plan_costs)

        
        time += @elapsed B, fire_plan_costs, num_plans_per_fire  = 
                add_suppression_plans(fire_data, p, r, fire_subproblem_models, num_plans_per_fire, B, fire_plan_costs, 0.01)
        
        total_crew_plans = sum(num_plans_per_crew)
        total_fire_plans = sum(num_plans_per_fire)

    end
    
    final_num_fire_plans = copy(num_plans_per_fire)
    final_num_crew_plans = copy(num_plans_per_crew)
    

    int_problem = @elapsed model, supp, fire_plans_chosen, crew_routes_chosen = 
                            solve_integer_MP(num_plans_per_crew, 
                            num_plans_per_fire, 
                            crew_plan_costs, 
                            fire_plan_costs,
                            A, B);
    return_time = @elapsed d = Dict("obj_LR" => objective_value(m), "obj_MP" => objective_value(mp_model), 
         "obj_PB" => objective_value(model),
         "time_LR" => relaxation_time, "time_formulate" => formulate_time, "time_fire_warm_start" => fire_warm_start_time, 
         "time_crew_warm_start" => crew_warm_start_time, "iterations" => n_iter, "cg_time" => time,
         "restore_int_time" => int_problem
         "fire_plans_chosen" => fire_plans_chosen, "crew_routes_chosen" => crew_routes_chosen,
         "initial_fire_plans" => initial_num_fire_plans, "initial_crew_routes" => initial_num_crew_plans,
         "final_fire_plans" => final_num_fire_plans, "final_crew_routes" => final_num_crew_plans,
         "supp_LR" => value.(l) / LINE_PER_CREW, "supp_PB" => supp, "num_plans_per_crew" => num_plans_per_crew,
         "num_plans_per_fire" => num_plans_per_fire, "crew_plan_costs" => crew_plan_costs, 
         "fire_plan_costs" => fire_plan_costs, "A" => A, "B" => B)
    return d
end

test_cg_time (generic function with 1 method)

In [54]:
timings = []

Any[]

In [62]:
limit = 1800

1800

In [63]:
crews = [20, 25, 30]
fires = [20, 25, 30];

In [64]:
for crew in crews
    for fire in fires
        NUM_CREWS = crew
        NUM_FIRES = fire
        start_perims, progressions, fire_dists, base_fire_dists, tau, tau_base_to_fire, rest_by, current_fire, rested_periods =
             read_inputs(in_path, init_path)

        arcs = define_arc_sets(tau, tau_base_to_fire, current_fire, rest_by, rested_periods)
        fire_progs = FireProgressionData(start_perims, progressions)
        
        d = test_cg_time(arcs, fire_progs, 1, 20, limit)
        true_time = (d["time_LR"] + d["time_fire_warm_start"] + d["time_crew_warm_start"] + d["time_formulate"] + d["cg_time"] 
        + d["restore_int_time"])
        
        to_optimality, feasible_solution, lb, ub, gap, formulate_time, solve_time = test_gurobi_time(arcs, fire_progs, maximum([true_time, limit]))
        
        push!(timings, copy([(to_optimality, feasible_solution, lb, ub, gap, formulate_time, solve_time),
                 (d["time_LR"], d["time_fire_warm_start"], d["time_crew_warm_start"], d["time_formulate"],
                  d["cg_time"], d["restore_int_time"], d["obj_PB"])]))
    end
end

In [65]:
timings

18-element Vector{Any}:
 Tuple{Real, Real, Float64, Float64, Float64, Float64, Float64}[(true, true, 9.864700630240481e6, 9.865654575218208e6, 9.669353112392829e-5, 34.6577808, 54.4274217), (35.1403483, 0.7059322, 39.1470167, 30.4855556, 41.230449599999986, 1.4775433, 1.0074170510050012e7)]
 Tuple{Real, Real, Float64, Float64, Float64, Float64, Float64}[(false, true, 2.009344665175299e7, 2.0096575590210617e7, 0.00015569510554579302, 115.2775525, 1086.092535), (129.1836738, 1.1031136, 114.5847021, 97.6820908, 111.00553269999999, 23.9362549, 2.0782143410384215e7)]
 Tuple{Real, Real, Float64, Float64, Float64, Float64, Float64}[(false, true, 2.9184207606317412e7, 2.9187811315352328e7, 0.00012346623033774784, 268.590304, 932.5448877), (271.4639254, 0.6850069, 293.5175749, 267.654553, 187.3184130000001, 6.8034636, 2.932235636724548e7)]
 Tuple{Real, Real, Float64, Float64, Float64, Float64, Float64}[(true, true, 7.391116161382975e6, 7.39129983863049e6, 2.4850466294840665e-5, 51.0822236, 93.0

In [59]:
crews = [20, 25, 30]
fires = [10, 15, 20]
old_timings = copy(timings)

9-element Vector{Any}:
 Tuple{Real, Real, Float64, Float64, Float64, Float64, Float64}[(true, true, 9.864700630240481e6, 9.865654575218208e6, 9.669353112392829e-5, 34.6577808, 54.4274217), (35.1403483, 0.7059322, 39.1470167, 30.4855556, 41.230449599999986, 1.4775433, 1.0074170510050012e7)]
 Tuple{Real, Real, Float64, Float64, Float64, Float64, Float64}[(false, true, 2.009344665175299e7, 2.0096575590210617e7, 0.00015569510554579302, 115.2775525, 1086.092535), (129.1836738, 1.1031136, 114.5847021, 97.6820908, 111.00553269999999, 23.9362549, 2.0782143410384215e7)]
 Tuple{Real, Real, Float64, Float64, Float64, Float64, Float64}[(false, true, 2.9184207606317412e7, 2.9187811315352328e7, 0.00012346623033774784, 268.590304, 932.5448877), (271.4639254, 0.6850069, 293.5175749, 267.654553, 187.3184130000001, 6.8034636, 2.932235636724548e7)]
 Tuple{Real, Real, Float64, Float64, Float64, Float64, Float64}[(true, true, 7.391116161382975e6, 7.39129983863049e6, 2.4850466294840665e-5, 51.0822236, 93.01

In [73]:
(1007 - 986.5) /986.5

0.020780537252914344

In [87]:
sum(timings[17][2][1:6]), sum(timings[17][1][6:7])

(2669.0767982999996, 2674.0314115)

In [88]:
(timings[17][2][7] - timings[17][1][4]) /  timings[17][1][4] * 100

1.2186412251474623

In [90]:
timings[17][1][4]

3.3219083499886304e7

In [129]:
to_optimality, feasible_solution, lb, ub, gap, formulate_time, solve_time = test_gurobi_time(arcs, fire_progs, maximum([true_time, limit]))

(true, true, 1.9253172996207774e6, 1.9255037178859836e6, 9.68153234266111e-5, 2.1375163, 3.7449978)

In [99]:
arr = hcat(result["crew_plan_histories"]...)
npzwrite("data/output/num_routes_history_20220428.npz", arr)

arr = hcat(result["fire_plan_histories"]...)
npzwrite("data/output/num_plans_history_20220428.npz", arr)

npzwrite("data/output/routes_chosen_20220428.npz", result["crew_routes_chosen"])
npzwrite("data/output/plans_chosen_20220428.npz", result["fire_plans_chosen"])

npzwrite("data/output/progress_20220428.npz", convert.(Float64, result["progress"]))

In [99]:
function unsuppressed_damage(current_perim, progs)
    
    perims = zeros(length(progs))
    perims[1] = current_perim * progs[1]
    for i in 2:length(progs)
        perims[i] = perims[i-1] * progs[i]
    end

    sum(perims) - perims[1] / 2 - perims[length(progs)] / 2
end

unsuppressed_damage (generic function with 1 method)

In [100]:
function myopic_unsuppressed_damage(current_perim, progs)
    return current_perim * (1 + progs[1])/2
end 

myopic_unsuppressed_damage (generic function with 1 method)

In [101]:
function unsuppressed_size(current_perim, progs)
    
    perims = zeros(length(progs))
    perims[1] = current_perim * progs[1]
    for i in 2:length(progs)
        perims[i] = perims[i-1] * progs[i]
    end
    perims[length(progs)]
end

unsuppressed_size (generic function with 1 method)

In [102]:
function myopic_unsuppressed_size(current_perim, progs)
    current_perim
end

myopic_unsuppressed_size (generic function with 1 method)

In [103]:
function baseline(func)
    
    rest_periods = [[rest_by[i] + j for j = 0:BREAK_LENGTH if rest_by[i] + 1 + j <= NUM_TIME_PERIODS] for i = 1:NUM_CREWS]

    current_period = 0
    current_locations = copy(current_fire)
    current_perims = copy(start_perims)
    current_progs = copy(progressions)
    all_assignments = []

    while (current_period < NUM_TIME_PERIODS)
        # get priorities
        priorities = [func(current_perims[f], current_progs[f, :]) for f = 1:NUM_FIRES] 
        if sum(priorities) > 0.00001
            priorities = priorities / sum(priorities)
        end

        # get available crews
        available_crews = [crew for crew in 1:NUM_CREWS if !(current_period+1 in rest_periods[crew])]

        # make distance array
        dists = zeros(NUM_CREWS, NUM_FIRES)
        for crew in 1:NUM_CREWS
            if current_locations[crew] > -1
                dists[crew, :] = fire_dists[current_locations[crew], 1:NUM_FIRES]
            else
                dists[crew, :] = base_fire_dists[crew, 1:NUM_FIRES]
            end
        end


        # get allocations
        allocs = get_allocations(length(available_crews), priorities)
        assignments = get_assignments(available_crews, dists, allocs)


        # update variables
        current_period += 1
        new_locations = convert.(Int, zeros(NUM_CREWS)) .- 1
        for fire in 1:NUM_FIRES
            for crew in assignments[fire]
                new_locations[crew] = fire    
            end
        end
        for crew in 1:NUM_CREWS
            push!(all_assignments, (crew, current_locations[crew], new_locations[crew], current_period))
        end
        current_locations = copy(new_locations)
        current_perims = [(current_perims[fire] - length(assignments[fire]) * LINE_PER_CREW / 2) * 
                  current_progs[fire, 1] - length(assignments[fire]) * LINE_PER_CREW / 2 for fire=1:NUM_FIRES]
        current_perims = max.(current_perims, 0)
        if current_period > 1


            current_progs = current_progs[:, 2:size(current_progs)[2]]
        end
    end
    
    supp = convert.(Int, zeros(NUM_FIRES, NUM_TIME_PERIODS))
    for assignment in all_assignments
        if (assignment[3] > 0)
            supp[assignment[3], assignment[4]] += 1
        end
    end
    
    base_fire_cost = 
    sum([base_fire_dists[i[1], i[3]] for i in all_assignments if (i[2] == -1) & (i[3] > -1)]) + 
    sum([ALPHA + base_fire_dists[i[1], i[2]] for i in all_assignments if (i[3] == -1) & (i[2] > -1)])

    fire_fire_cost = sum([ALPHA + fire_dists[i[2], i[3]] for i in all_assignments if (i[3] > -1) & (i[2] > -1)])

    total_route_cost = base_fire_cost + fire_fire_cost

    total_damage_cost = BETA * sum(fire_plan_cost(start_perims[fire], supp[fire, :] * LINE_PER_CREW, progressions[fire, :])
                                      for fire in 1:NUM_FIRES)

    total_cost = total_route_cost + total_damage_cost
    
    return supp, total_cost
end

baseline (generic function with 1 method)

In [104]:
function get_assignments(available_crews, distances, allocations)
    
    priority = reverse(sortperm(allocations))
    d = Dict()
    
    for fire in priority
        dists = distances[:, fire]
        assignments = sortperm(dists)
        assignments = [i for i in assignments if i in available_crews][1:allocations[fire]]
        available_crews = [i for i in available_crews if !(i in assignments)]
        d[fire] = assignments
    end
    
    d
end   

get_assignments (generic function with 1 method)

In [105]:
function get_allocations(available_crews, proportions)
    
    priority = reverse(sortperm(proportions))
    allocations = convert.(Int, zeros(length(proportions)))
    
    for fire in priority
        allocations[fire] = round(available_crews * proportions[fire])
        proportions[fire] = 0
        if sum(proportions) > 0.00001
            proportions = proportions / sum(proportions)
        end
        available_crews -= allocations[fire]
    end
    
    allocations
end

get_allocations (generic function with 1 method)